In [40]:
import pandas as pd
base = pd.read_csv('credit_data.csv')
base = base.drop('i#clientid', axis = 1)
base = base.dropna()
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
X = base.drop('c#default', axis = 1)
y = base['c#default']
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [41]:
neuronios_entrada = 3
neuronios_oculta = 2
neuronios_saida = neuronios_entrada
# 3 -> 2 -> 3

In [42]:
import tensorflow as tf

In [43]:
xph = tf.placeholder(tf.float32, shape = [None, neuronios_entrada])

In [44]:
from tensorflow.contrib.layers import fully_connected
camada_oculta = fully_connected(inputs = xph, num_outputs = neuronios_oculta, activation_fn = None)
camada_saida = fully_connected(inputs = camada_oculta, num_outputs = neuronios_saida)

In [45]:
erro = tf.losses.mean_squared_error(labels = xph, predictions = camada_saida)
otimizador = tf.train.AdamOptimizer(0.01)
treinamento = otimizador.minimize(erro)

In [46]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(1000):
        custo, _ = sess.run([erro, treinamento], feed_dict = {xph: X})
        if epoca % 100 == 0:
            print('erro: ' + str(custo))
    X2d_encode = sess.run(camada_oculta, feed_dict = {xph: X})
    X3d_decode = sess.run(camada_saida, feed_dict = {xph: X})

erro: 1.822283
erro: 0.6850898
erro: 0.6190646
erro: 0.6165702
erro: 0.6158049
erro: 0.6155921
erro: 0.61549306
erro: 0.6151795
erro: 0.6147627
erro: 0.61437577


In [47]:
X2d_encode.shape

(1997, 2)

In [48]:
X3d_decode.shape

(1997, 3)

In [49]:
X2 = scaler_x.inverse_transform(X)
X2

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [50]:
X3d_decode2 = scaler_x.inverse_transform(X3d_decode)
X3d_decode2

array([[6.8575930e+04, 4.0807560e+01, 6.5928110e+03],
       [4.5333863e+04, 4.0807560e+01, 4.4454878e+03],
       [6.0801395e+04, 4.0807560e+01, 5.6168853e+03],
       ...,
       [4.5333863e+04, 4.0807560e+01, 6.0554517e+03],
       [4.6186484e+04, 5.8964218e+01, 4.4454878e+03],
       [7.1049211e+04, 4.0807560e+01, 6.5309468e+03]], dtype=float32)

In [51]:
from sklearn.metrics import mean_absolute_error
mae_income = mean_absolute_error(X2[:,0], X3d_decode2[:,0])
mae_income

7076.3921286381

In [52]:
mae_age = mean_absolute_error(X2[:,1], X3d_decode2[:,1])
mae_age

10.078172138235011

In [53]:
mae_loan = mean_absolute_error(X2[:,2], X3d_decode2[:,2])
mae_loan

1784.8301553867298

In [54]:
X_encode = pd.DataFrame({'atributo1': X2d_encode[:,0], 'atributo2': X2d_encode[:,1], 'classe': y})

In [55]:
X_encode.head()

,atributo1,atributo2,classe
0,1.486158,1.373214,0
1,-0.161346,-1.120568,0
2,0.934117,0.843253,0
3,0.187720,-0.523579,0
4,2.692830,0.218876,1


In [56]:
import tensorflow as tf
colunas = [tf.feature_column.numeric_column(key = column) for column in X_encode.columns]
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_encode, y, test_size = 0.3, random_state = 0)
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento,
                                                        y = y_treinamento,
                                                        batch_size = 8, 
                                                        num_epochs = None,
                                                        shuffle = True)
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4])
classificador.train(input_fn = funcao_treinamento, steps = 1000)
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, y = y_teste,
                                              batch_size = 8, num_epochs = 1000,
                                              shuffle = False)
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/7f/wlmrltf57q95bdd5fl85dpw40000gn/T/tmpiup4xlcq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c35681208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Che

In [57]:
metricas_teste

{'accuracy': 1.0,
 'accuracy_baseline': 0.8635,
 'auc': 1.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.0016968956,
 'label/mean': 0.1365,
 'loss': 0.013575165,
 'precision': 1.0,
 'prediction/mean': 0.13671768,
 'recall': 1.0,
 'global_step': 1000}